In [1]:
import os
import random
from PIL import Image, ImageOps, ImageDraw
import time
import math
import shutil

max_time = 30

def intersect(box1, box2):
    return not (box1[2] <= box2[0] or box1[0] >= box2[2] or box1[3] <= box2[1] or box1[1] >= box2[3])

def transform(image):
    
    angle = random.uniform(-30, 30)
    # Randomly rotate the image
    image = image.rotate(angle, resample=Image.BICUBIC)

     # Randomly flip the image
    if random.random() > 0.5:
        image = ImageOps.mirror(image)
        
    return image

def transform_box(box, angle):
    
    width =  box[2] - box[0]
    height = box[3] - box[1]

    # Get rotation angle and apply rotation to box coordinates
    angle = angle
    cos_val = abs(math.cos(math.radians(angle)))
    sin_val = abs(math.sin(math.radians(angle)))
    new_x1 = (box[0] - width / 2) * cos_val - (box[1] - height / 2) * sin_val + width / 2
    new_y1 = (box[0] - width / 2) * sin_val + (box[1] - height / 2) * cos_val + height / 2
    new_x2 = (box[2] - width / 2) * cos_val - (box[3] - height / 2) * sin_val + width / 2
    new_y2 = (box[2] - width / 2) * sin_val + (box[3] - height / 2) * cos_val + height / 2

    # Randomly flip the box
    if random.random() > 0.5:
        new_x1, new_x2 = width - new_x2, width - new_x1

    return [new_x1, new_y1, new_x2, new_y2]

def lblformat(box, background_size):
    labelimg_box = [
    (box[0]/ float(background_size)) + ((box[2]-box[0])/ float(background_size))*0.5,#(box1[0]/ float(background_size))*0.5,  # xmin
    (box[1]/ float(background_size))+ ((box[3]-box[1])/ float(background_size))*0.5,#(box1[1]/ float(background_size))*0.5,  # ymin
    (box[2]-box[0])/ float(background_size),  # width
    (box[3]-box[1])/ float(background_size),  # height
    ]
    return labelimg_box

def detect_class(name):
    if 'cardboard' in name:
        return 0
    elif 'glass' in name:
        return 1
    elif 'metal' in name:
        return 2
    elif 'paper' in name:
        return 3
    elif 'plastic' in name:
        return 4
    else:
        return 5

In [2]:
background_size = 1000

directory = 'png_train'

output_dir = "new_images_train"  # replace with your directory path

folder_count = len([item for item in os.listdir(output_dir) if os.path.isdir(os.path.join(output_dir , item))])

folder_name = f'exp{folder_count+1}'

new_folder = os.path.join(output_dir, f'exp{folder_count+1}')
os.mkdir(new_folder)

output_dir = new_folder

#copy the classes.txt file to the new folder
source_file = '/Users/casper/Desktop/pytorch/dataset_generator/new_images_train/classes.txt'
filename = os.path.basename(source_file)
destination_file = os.path.join(output_dir, filename)
shutil.copy2(source_file, destination_file)

boxes = []


for i in range (1, 1000):
    
    images = [filename for filename in os.listdir(directory) if filename.endswith('.png')]
    bgs = [filename for filename in os.listdir('bgs') if filename.endswith('.png')]
    
    selected_images = random.sample(images, 5)
    random_bg = Image.open(os.path.join('bgs', random.choice(bgs)))
    
    image_names = [os.path.splitext(image)[0] for image in selected_images]
    
    new_image = random_bg.resize((background_size, background_size))
    
    img1 = Image.open(os.path.join(directory, selected_images[0]))
    img1 = img1.convert("RGBA")
    img1 = transform(img1)
    
    scale_factor = random.uniform(0.5, 1)
    
    new_size = tuple(int(dim * scale_factor) for dim in img1.size)
    img1 = img1.resize(new_size)
    
    # Get the bounding box of the non-transparent part of the image
    bbox1 = img1.getbbox()

    # Calculate the width and height of the non-transparent part of the image at the given scale
    width = (bbox1[2] - bbox1[0])
    height = (bbox1[3] - bbox1[1])
    
    x1 = random.uniform(0, background_size - width)
    y1 = random.uniform(0, background_size - height)
    
    box1 = (x1, y1, x1 + width, y1 + height)
    boxes.append(box1)
    
    new_image.paste(img1, (int(x1-bbox1[0]), int(y1-bbox1[1])), img1)

    # Draw rectangles on new image based on box coordinates
    # draw = ImageDraw.Draw(new_image)
    # draw.rectangle(box1, outline="blue")
    
# draw the second box at a random position that does not overlap with the first box
    start_time = time.time()
    placed = False
    
    while not placed:
        
        img2 = Image.open(os.path.join(directory, selected_images[1]))
        img2 = img2.convert("RGBA")
        img2 = transform(img2)
        
        scale_factor = random.uniform(0.5, 1)
            
        new_size = tuple(int(dim * scale_factor) for dim in img2.size)
        img2 = img2.resize(new_size)
        
        # Get the bounding box of the non-transparent part of the image
        bbox2 = img2.getbbox()

        # Calculate the width and height of the non-transparent part of the image at the given scale
        width = (bbox2[2] - bbox2[0])
        height = (bbox2[3] - bbox2[1])
        
        x2 = random.uniform(0, background_size - width)
        y2 = random.uniform(0, background_size - height)
        
        box2 = (x2, y2, x2 + width, y2 + height)
        boxes.append(box2)

        if not any([intersect(box2, box1)]):
            
            new_image.paste(img2, (int(x2-bbox2[0]), int(y2-bbox2[1])), img2)

            # Draw rectangles on new image based on box coordinates
            # draw = ImageDraw.Draw(new_image)
            # draw.rectangle(box2, outline="red")
            
            break
        else:
            # print(f"{i} Retrying with another coordinate...2nd image")
            time.sleep(0.5)


# draw the third box at a random position that does not overlap with the first box
    start_time = time.time()
    placed = False
    
    while not placed:
        
        img3 = Image.open(os.path.join(directory, selected_images[2]))
        img3 = img3.convert("RGBA")
        img3 = transform(img3)
    
        scale_factor = random.uniform(0.5, 1)
            
        new_size = tuple(int(dim * scale_factor) for dim in img3.size)
        img3 = img3.resize(new_size)
        
        # Get the bounding box of the non-transparent part of the image
        bbox3 = img3.getbbox()

        # Calculate the width and height of the non-transparent part of the image at the given scale
        width = (bbox3[2] - bbox3[0])
        height = (bbox3[3] - bbox3[1])
        
        x3 = random.uniform(0, background_size - width)
        y3 = random.uniform(0, background_size - height)
        
        box3 = (x3, y3, x3 + width, y3 + height)
        boxes.append(box3)

        if not any([intersect(box3, box1), intersect(box3, box2)]):
            
            new_image.paste(img3, (int(x3-bbox3[0]), int(y3-bbox3[1])), img3)

            # Draw rectangles on new image based on box coordinates
            # draw = ImageDraw.Draw(new_image)
            # draw.rectangle(box3, outline="green")
            
            break
        else:
            # print(f"{i} Retrying with another coordinate...3rd image")
            time.sleep(0.5)    
                
# draw the fourth box at a random position that does not overlap with the first box
    start_time = time.time()
    placed = False
    
    while not placed:
        
        img4 = Image.open(os.path.join(directory, selected_images[3]))
        img4 = img4.convert("RGBA")
        img4 = transform(img4)
    
        scale_factor = random.uniform(0.5, 1)
            
        new_size = tuple(int(dim * scale_factor) for dim in img4.size)
        img4 = img4.resize(new_size)
        
        # Get the bounding box of the non-transparent part of the image
        bbox4 = img4.getbbox()

        # Calculate the width and height of the non-transparent part of the image at the given scale
        width = (bbox4[2] - bbox4[0])
        height = (bbox4[3] - bbox4[1])
        
        x4 = random.uniform(0, background_size - width)
        y4 = random.uniform(0, background_size - height)
        
        box4 = (x4, y4, x4 + width, y4 + height)
        boxes.append(box4)

        if not any([intersect(box4, box1), intersect(box4, box2), intersect(box4, box3)]):
            
            new_image.paste(img4, (int(x4-bbox4[0]), int(y4-bbox4[1])), img4)

            # Draw rectangles on new image based on box coordinates
            # draw = ImageDraw.Draw(new_image)
            # draw.rectangle(box4, outline="blue")
            
            break
        else:
            # print(f"{i} Retrying with another coordinate...4th image")
            time.sleep(0.5)    
            
# draw the fourth box at a random position that does not overlap with the first box
    start_time = time.time()
    placed = False
    
    while not placed:
        
        img5 = Image.open(os.path.join(directory, selected_images[4]))
        img5 = img5.convert("RGBA")
        img5 = transform(img5)
    
        scale_factor = random.uniform(0.5, 1)
            
        new_size = tuple(int(dim * scale_factor) for dim in img5.size)
        img5 = img5.resize(new_size)
        
        # Get the bounding box of the non-transparent part of the image
        bbox5 = img5.getbbox()

        # Calculate the width and height of the non-transparent part of the image at the given scale
        width = (bbox5[2] - bbox5[0])
        height = (bbox5[3] - bbox5[1])
        
        x5 = random.uniform(0, background_size - width)
        y5 = random.uniform(0, background_size - height)
        
        box5 = (x5, y5, x5 + width, y5 + height)
        boxes.append(box5)

        if not any([intersect(box5, box1), intersect(box5, box2), intersect(box5, box3), intersect(box5, box4)]):
            
            new_image.paste(img5, (int(x5-bbox5[0]), int(y5-bbox5[1])), img5)

            # Draw rectangles on new image based on box coordinates
            # draw = ImageDraw.Draw(new_image)
            # draw.rectangle(box4, outline="blue")
            
            break
        else:
            # print(f"{i} Retrying with another coordinate...5th image")
            time.sleep(0.5)    
                
    new_image_name = f"{i}_{'_'.join(image_names)}_{folder_name}.jpg"
    new_image_path = os.path.join(output_dir, new_image_name)
    new_image.save(new_image_path)
    print(f"{new_image_name} saved")
    
    new_txt_name = f"{output_dir}/{i}_{'_'.join(image_names)}_{folder_name}.txt"
    
    labelimg_box = lblformat(box1, background_size)
    text1 = f"{detect_class(image_names[0])} {labelimg_box[0]} {labelimg_box[1]} {labelimg_box[2]} {labelimg_box[3]}"
    
    labelimg_box = lblformat(box2, background_size)
    text2 = f"{detect_class(image_names[1])} {labelimg_box[0]} {labelimg_box[1]} {labelimg_box[2]} {labelimg_box[3]}"
    
    labelimg_box = lblformat(box3, background_size)
    text3 = f"{detect_class(image_names[2])} {labelimg_box[0]} {labelimg_box[1]} {labelimg_box[2]} {labelimg_box[3]}"
    
    labelimg_box = lblformat(box4, background_size)
    text4 = f"{detect_class(image_names[3])} {labelimg_box[0]} {labelimg_box[1]} {labelimg_box[2]} {labelimg_box[3]}"
    
    labelimg_box = lblformat(box5, background_size)
    text5 = f"{detect_class(image_names[4])} {labelimg_box[0]} {labelimg_box[1]} {labelimg_box[2]} {labelimg_box[3]}"

    with open(f"{new_txt_name}", "w") as file:
        file.write(text1 + '\n')
        file.write(text2 + '\n')
        file.write(text3 + '\n')
        file.write(text4 + '\n')
        file.write(text5 + '\n')

1_glass384_glass12_plastic183_cardboard18_paper499_exp8.jpg saved
2_paper43_paper6_paper380_metal131_glass420_exp8.jpg saved
3_metal285_plastic168_trash38_trash48_glass388_exp8.jpg saved
4_glass60_paper432_metal93_glass345_plastic94_exp8.jpg saved
5_metal215_plastic313_paper66_glass496_paper96_exp8.jpg saved
6_paper523_plastic357_glass28_paper283_glass477_exp8.jpg saved
7_plastic393_plastic399_glass183_paper413_paper417_exp8.jpg saved
8_trash33_plastic6_trash132_plastic377_glass6_exp8.jpg saved
9_paper207_paper226_plastic253_paper2_paper274_exp8.jpg saved
10_plastic191_glass78_plastic14_glass304_glass434_exp8.jpg saved
11_plastic90_metal364_paper219_paper526_glass27_exp8.jpg saved
12_plastic190_plastic325_metal1_metal364_glass321_exp8.jpg saved
13_plastic315_metal209_paper183_metal211_cardboard168_exp8.jpg saved
14_glass480_trash49_paper260_glass286_trash147_exp8.jpg saved
15_plastic263_glass390_plastic189_paper304_paper358_exp8.jpg saved
16_paper290_trash72_glass145_glass181_glass1_ex